In [1]:
import pandas as pd

In [35]:
df = pd.read_csv("log_fw_3.csv", sep=";", header=0)
df = df.sample(n = 100000)

In [36]:
df

,datetime,ipsrc,ipdst,proto,portsrc,portdst,policyid,action,numproto
677715,23/02/2023 01:06,109.234.162.235,17.17.17.17,TCP,56642.0,60758.0,555.0,PERMIT,1.0
473135,20/02/2023 07:52,50.46.240.212,17.17.17.17,TCP,65293.0,443.0,1.0,PERMIT,6.0
776846,25/02/2023 06:42,89.248.165.30,17.17.17.17,TCP,56081.0,3487.0,999.0,DENY,6.0
917570,28/02/2023 12:35,198.46.189.193,17.17.17.17,TCP,54694.0,55423.0,999.0,DENY,6.0
191313,15/02/2023 01:01,109.234.162.235,17.17.17.17,TCP,41410.0,60916.0,555.0,PERMIT,1.0
...,...,...,...,...,...,...,...,...,...
932845,28/02/2023 12:37,198.46.189.193,17.17.17.17,TCP,55752.0,56749.0,999.0,DENY,6.0
568192,22/02/2023 01:00,109.234.162.235,17.17.17.17,TCP,51206.0,55014.0,555.0,PERMIT,1.0
265618,17/02/2023 02:00,50.219.89.18,17.17.17.17,TCP,18371.0,443.0,1.0,PERMIT,6.0
15300,12/02/2023 18:33,152.32.202.139,17.17.17.17,TCP,34471.0,7890.0,999.0,DENY,6.0


In [37]:
df_tcp = df[df['proto'] == 'TCP']

In [38]:
#Fournir un rapprochement des règles (policyid) par rapport aux ports de destination et les action. Cette analyse portera uniquement sur le protocole par défaut Tcp.
grouped_rules = df_tcp.groupby(['portdst', 'action', 'policyid'])['policyid'].count().reset_index(name='count')
grouped_rules["policyid"] = grouped_rules["policyid"].values.astype(str)
grouped_rules["portdst"] = grouped_rules["portdst"].values.astype(str)



In [39]:
grouped_rules.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14950 entries, 0 to 14949
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   portdst   14950 non-null  object
 1   action    14950 non-null  object
 2   policyid  14950 non-null  object
 3   count     14950 non-null  int64 
dtypes: int64(1), object(3)
memory usage: 467.3+ KB


In [40]:
#make a butifull graphe plotly of grouped_rules df, change before the type of policyid in string
import plotly.express as px
fig = px.bar(grouped_rules, x='portdst', y='count', color='action', barmode='group', height=400)
fig.show()
